# Using Web Search as a tool

[Web Search](https://dev.writer.com/home/web-search-tool) is Writer's built-in tool for retrieving up-to-date information from the internet directly in a chat with a Palmyra LLM.

This cookbook shows how to use the Web Search tool to answer real-time questions and customize search behavior during a chat with an AI assistant.

## Prerequisites
Before getting started, you'll need:

- A [Writer AI Studio](https://app.writer.com/register) account
- An API key, which you can obtain by following the [API Quickstart](https://dev.writer.com/home/quickstart)

## Setup
Install the Writer SDK:

In [ ]:
%pip install writer-sdk python-dotenv -q

Next, set the `WRITER_API_KEY` environment variable. We recommend setting it in a `.env` file in the root of your project, but this tutorial will set it in an environment variable if you don't have a `.env` file.

In [ ]:
import getpass
import os
from writerai import Writer

if not os.getenv("WRITER_API_KEY"):
    os.environ["WRITER_API_KEY"] = getpass.getpass("Enter your Writer API key: ")

client = Writer()  # The Writer client automatically uses the WRITER_API_KEY environment variable to authenticate.

## Usage

The Web Search tool follows this format:

```json
{
    "type": "web_search",
    "function": {
        "include_domains": ["example.com"],
        "exclude_domains": ["example2.com"]
    }
}
```
The `include_domains` field allows you to restrict results to one or more specific domains.

The `exclude_domains` field allows you to filter out results from unwanted domains.


In [ ]:
tools = [{
    "type": "web_search",
    "function": {
        "include_domains": ["wunderground.com"],
        "exclude_domains": ["accuweather.com"]
    }
}]

The next code snippet asks you to input a question to answer using the Web Search tool, and then sends that question to the LLM. It passes the Web Search tool to the LLM.

In [ ]:
messages = [
    {
        "role": "user",
        "content": (
            "Provide the latest weather information for New York from September 20th, 2025 to September 27th, 2025. "
            "Return the data day by day, including only: date, high temperature, low temperature, "
            "and average temperature."
        )
    }
]

response = client.chat.chat(
    model="palmyra-x5",
    messages=messages,
    tools=tools,  # The tools array defined earlier.
    tool_choice="auto"
)
print(response.choices[0].message.content)

## Streaming Web Search Response
You can stream the chat response from the Web Search tool in real-time. The following snippet streams the response as it is generated, collects the text, and prints it incrementally.

In [ ]:
response_stream = client.chat.chat(
    model="palmyra-x-004",
    messages=messages,
    tools=tools,
    tool_choice="auto",
    stream=True
)

output_text = ""

for chunk in response_stream:
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content, end="", flush=True)
        output_text += chunk.choices[0].delta.content

print(f"Final output: {output_text}")

## Limiting the number of sources
You can ask the LLM to return results from a specific number of sources. 
This is useful when you want concise information with links to only a few reliable sources.

In [ ]:
messages = [
    {
        "role": "user",
        "content": (
            "Find the latest breakthroughs in quantum computing. "
            "Limit the results to only 3 sources and provide links to each source."
        )
    }
]

tools = [{
    "type": "web_search",
    "function": {}
}]

response = client.chat.chat(
    model="palmyra-x5",
    messages=messages,
    tools=tools,
    tool_choice="auto"
)

print(response.choices[0].message.content)

## Including Raw Source Text

You can instruct the LLM to include the raw text from the sources it uses. 
This is helpful if you want to see the original content for verification or analysis.

In [ ]:
messages = [
    {
        "role": "user",
        "content": "Find recent news articles about electric vehicles published within the last 7 days. "
        "Summarize the key points from each article and include the source links."
    }
]

tools = [{"type": "web_search", "function": {}}]

response = client.chat.chat(
    model="palmyra-x5",
    messages=messages,
    tools=tools,
    tool_choice="auto"
)

# Access raw content for each source
sources = response.choices[0].message.web_search_data.sources
for source in sources:
    print(
        f"raw_content: {source.raw_content[:20]}..., url: {source.url[:20]}..."
    )
print(response.choices[0].message.content)
    